In [2]:
import pandas as pd

In [3]:
#Find your user_id by logging into your account, going to your profile, going to your workouts, and get the link from "Download Workout"
import requests
from config import user_id, username, password
s = requests.Session()
payload = {'username_or_email':username, 'password':password}
s.post('https://api.onepeloton.com/auth/login', json=payload)
download = f'https://api.onepeloton.com/api/user/{user_id}/workout_history_csv?timezone=America/Chicago'
response = s.get(download)
with open('C:/Users/erdos/Class Folder/Peloton_Data/Resources/input_workouts.csv', 'wb') as file:
    file.write(response.content)

In [4]:
#read the imported CSV file into a dataframe
df = pd.read_csv('Resources/input_workouts.csv')

In [5]:
# convert the column of time objects to datetime objects
df['date'] = df['Workout Timestamp'].str.slice(stop=-12)
df['Workout Timestamp'] = df['date'].apply(pd.to_datetime)
df.drop(['date','Class Timestamp','Avg. Incline','Avg. Pace (min/mi)'], axis=1, inplace=True)
df.fillna(0, inplace=True)

#convert float to int for columns that will always be whole numbers. 
df[['Total Output', 'Avg. Watts', 'Avg. Cadence (RPM)']] = df[['Total Output', 'Avg. Watts', 'Avg. Cadence (RPM)']].astype(int)

## Change the threshold date and FTP Score to match with your data.

In [6]:
#Setup initial FTP score: 
threshold_date = datetime.today().strftime('%Y-%m-%d')
df['Current FTP'] = 0

# set the value of 'Current FTP' 
df.loc[df['Workout Timestamp'] <= threshold_date, 'Current FTP'] = <your FTP score> 

# ----- Uncomment when you want to add a new FTP score-----#

def add_current_ftp(df, new_ftp, start_date, end_date):
    # Set the value of 'Current FTP' to the input value for the rows where the date is within the input date range
    df.loc[(df['Workout Timestamp'] >= start_date) & (df['Workout Timestamp'] <= end_date), 'Current FTP'] = new_ftp

    # Print the resulting DataFrame
    print(df)

# Add the date range during which the new FTP score was documented
add_current_ftp(df, <new_ftp_value>, pd.to_datetime(<start_date>), pd.to_datetime(<end_date>))



In [ ]:
#move the columns to a place that makes sense. 
ftp_column = df.pop('Current FTP')
df.insert(8, 'Current FTP', ftp_column)

# create a fitness indicator column by dividing the avg watts by the avg heartrate for Cycling only 
cycling_mask = df['Fitness Discipline'] == 'Cycling'
fitness_indicator = df.loc[cycling_mask, 'Avg. Watts'] / df.loc[cycling_mask, 'Avg. Heartrate']
df.loc[cycling_mask, 'Fitness Indicator'] = fitness_indicator.round(2)

# Create a column that will assign avg power zones for your cycling workout. 
watt_ranges = [(0,0,''), (1, 55, 'Active Recovery'), (56, 75, 'Endurance'), (76, 90, 'Tempo'), (91, 105, 'Threshold'), (106, 120, 'VO2 Max')]

for index, row in df.iterrows():
    # get the current FTP and average watts for this row
    current_ftp = row['Current FTP']
    avg_watts = row['Avg. Watts']
    # calculate the percentage of the FTP score
    ftp_percentage = avg_watts / current_ftp * 100
    # loop through the watt ranges and categorize the average watts
    for watt_range in watt_ranges:
        if watt_range[0] <= ftp_percentage <= watt_range[1]:
            # assign the power zone to the 'Avg. Power Zone' column
            df.at[index, 'Avg. Power Zone'] = watt_range[2]
            break

df.head(5)

In [76]:
df.to_csv('Resources/output_workouts.csv')